### Imports e Constantes

In [1]:
import math
import random
import json
import numpy as np
import pandas as pd
import multiprocessing
import matplotlib.patheffects as PathEffects
from matplotlib         import pyplot as plt
from matplotlib.patches import Circle
from time import sleep, time
from multiprocessing.pool import ThreadPool, Pool
from tqdm import tqdm
#from pyswarm            import pso

#%matplotlib inline

In [9]:
# Map Informations
x_under_lim = 1
x_upper_lim = 44
y_under_lim = 1
y_upper_lim = 15

aps_positions = {'WAP11': [1.5 , 11.4],
                 'WAP5' : [33.0, 5.0 ],
                 'WAP3' : [16.5, 3.0 ],
                 'WAP1' : [4.3 , 5.0 ],
                 'WAP9' : [28.5, 12.0],
                 'WAP12': [4.0 , 17.5],
                 'WAP8' : [33.5, 12.0],
                 'WAP10': [20.0, 12.3],
                 'WAP6' : [38.0, 4.3 ],
                 'WAP7' : [39.0, 11.2],
                 'WAP4' : [23.0, 3.5 ],
                 'WAP2' : [8.0 , 6.0 ],
                 'WAP14': [17.8, 9.0 ],
                 'WAP15': [32.3, 9.0 ],
                 'WAP13': [13.5, 10.0]}

list_aps = list(aps_positions.keys())

#Constantes Log-Distance
d0  = 1
STD_DEV = 1
TXPOWER = 0

#PSO Infomations
dimension = 2
population = 10
generation = 10
fitness_criterion = 10e-8

### Comum para o PSO

In [3]:
def euclidian_distance(P1, P2):
    X1, Y1 = P1[0], P1[1]
    X2, Y2 = P2[0], P2[1]
    return round(math.sqrt((X2-X1)**2 + (Y2-Y1)**2),1)

def toPixels(x, y):
    return (x*100, y*100)

def generateMap(particles, costs, realPosition, currentRound):
    low_cost = 100000
    low_position = [0,0]
    
    perfPixels = toPixels(realPosition[0], realPosition[1])
    
    plt.clf()
    #plt.close()
    plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    plt.margins(0,0)                    
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    
    img = plt.imread("map.png")
    plt.imshow(img)
    
    txt = plt.text(perfPixels[0], perfPixels[1], "✖", weight='bold', fontsize=8, color="white", verticalalignment='center', horizontalalignment='center', zorder=5)
    txt.set_path_effects([PathEffects.withStroke(linewidth=2, foreground='black')])

    groups      = {}
    
    for i in range(len(particles)):
        pos = particles[i]
        posPixels = toPixels(pos[0], pos[1])
        label     = str(pos[0]) + str(pos[1])
        
        cost = costs[i]
        
        if cost < low_cost:
            low_cost = cost
            low_position = pos
        
        if label not in groups:
            groups[label] = [1, posPixels[0], posPixels[1], cost]
        else:
            groups[label][0] += 1
            
        circ = Circle((posPixels[0] + np.random.normal(0, 5), posPixels[1] + np.random.normal(0, 5)), radius=10, alpha=0.5, color="red", zorder=10)
        plt.gca().add_patch(circ)
    
    for group in groups.values():
        plt.annotate("Cost: " + str(round(group[3], 1)), xy=(group[1]+5, group[2]-5), size=2.7, va="center", ha="left", xytext=(group[1] + 50, group[2] - 80), zorder=8,
              bbox=dict(boxstyle="square", facecolor="#ffffffcc", edgecolor="#aaaaaa88", linewidth=0.4),
              arrowprops=dict(arrowstyle="-", antialiased=True, color="#444444", connectionstyle="arc3,rad=-0.2", linewidth=0.15))
        
    #Plot Low Position
    lowPosPixels = toPixels(low_position[0], low_position[1])
    circ = Circle((lowPosPixels[0], lowPosPixels[1]), radius=10, color="#2ca05aff", zorder=12)
    plt.gca().add_patch(circ)
    
    imgFilename = "sample-" + str(currentRound) + ".png"
    plt.savefig(imgFilename, dpi=300, bbox_inches="tight", pad_inches=0)

## Algoritmo 1

In [15]:
particles = {i:[round(random.uniform(x_under_lim, x_upper_lim),1), 
                round(random.uniform(y_under_lim, y_upper_lim),1), 0, 0, 0]
for i in range(population)}

In [14]:
particles

{0: [14.4, 5.8, 0, 0, 0],
 1: [14.3, 2.1, 0, 0, 0],
 2: [10.2, 8.6, 0, 0, 0],
 3: [13.8, 8.6, 0, 0, 0],
 4: [9.3, 14.6, 0, 0, 0],
 5: [39.8, 9.5, 0, 0, 0],
 6: [1.9, 11.0, 0, 0, 0],
 7: [42.1, 6.7, 0, 0, 0],
 8: [33.9, 11.8, 0, 0, 0],
 9: [28.2, 13.4, 0, 0, 0]}

In [25]:
def returnalgo(x):
    return [x*x,2]

vet = [1,2,3,4,5,6,7,8]

[returnalgo(p)[0] for p in vet]


[1, 4, 9, 16, 25, 36, 49, 64]

In [5]:
particles

[[28.1, 11.1],
 [22.7, 3.3],
 [28.2, 7.7],
 [10.1, 10.4],
 [24.4, 14.4],
 [7.9, 6.4],
 [8.0, 4.8],
 [16.8, 7.1],
 [26.0, 2.2],
 [31.6, 3.9],
 [6.0, 2.0],
 [39.7, 10.6],
 [4.1, 7.6],
 [42.6, 5.8],
 [41.0, 1.3],
 [9.3, 12.3],
 [13.1, 6.3],
 [25.7, 4.7],
 [43.5, 7.3],
 [1.1, 14.3],
 [36.0, 8.9],
 [27.3, 14.7],
 [11.4, 4.4],
 [6.1, 11.9],
 [21.7, 7.0],
 [7.6, 11.5],
 [16.1, 8.1],
 [33.8, 4.7],
 [13.6, 13.6],
 [2.2, 12.9],
 [22.4, 9.6],
 [40.3, 1.3],
 [21.6, 2.6],
 [34.3, 6.0],
 [29.3, 8.6],
 [22.8, 10.8],
 [16.4, 6.9],
 [32.3, 10.3],
 [42.3, 4.0],
 [41.0, 8.5],
 [7.5, 11.3],
 [13.2, 7.9],
 [15.8, 8.7],
 [32.8, 8.6],
 [36.9, 14.5],
 [30.5, 12.0],
 [34.4, 9.8],
 [20.9, 11.1],
 [5.8, 9.0],
 [33.2, 1.4]]

In [3]:
def update_velocity(particle, velocity, pbest, gbest, w_min=0.5, max=0.5, c=0.5):
    # Initialise new velocity array
    num_particle = len(particle)
    new_velocity = np.array([0.0 for i in range(num_particle)])
    
    # Randomly generate r1, r2 and inertia weight from normal distribution
    r1 = random.uniform(0,max)
    r2 = random.uniform(0,max)
    w = random.uniform(w_min,max)
    c1 = c
    c2 = c
    
    # Calculate new velocity
    for i in range(num_particle):
        new_velocity[i] = round((w*velocity[i] + c1*r1*(pbest[i]-particle[i])+c2*r2*(gbest[i]-particle[i])),0)
    
    return new_velocity

def update_position(particle, velocity):
    # Move particles by adding velocity
    new_particle = particle + velocity
    
    return new_particle

def pso_2d(population, dimension, generation, fitness_criterion, real_position, row):
    
    seed = time()
    random.seed(seed)
    #print('seed:', seed)
    
    # Population
    particles = [[round(random.uniform(x_under_lim, x_upper_lim),1), round(random.uniform(y_under_lim, y_upper_lim),1)] for i in range(population)]
    
    # Particle's best position
    pbest_position = particles
    
    # Fitness
    pbest_fitness = [fitness_function([p[0],p[1]], row) for p in particles]
    
    # Index of the best particle
    gbest_index = np.argmin(pbest_fitness)
    
    # Global best particle position
    gbest_position = pbest_position[gbest_index]
    
    # Velocity (starting from 0 speed)
    velocity = [[0.0 for j in range(dimension)] for i in range(population)]

    # Loop for the number of generation
    for t in range(generation):
        # Stop if the average fitness value reached a predefined success criterion
        if np.average(pbest_fitness) <= fitness_criterion:
            break
        else:
            for n in range(population):
                # Update the velocity of each particle
                velocity[n] = update_velocity(particles[n], velocity[n], pbest_position[n], gbest_position)
                
                # Move the particles to new position
                particles[n] = update_position(particles[n], velocity[n])

        # Calculate the fitness value
        pbest_fitness = [fitness_function([p[0],p[1]], row) for p in particles]
        
        # Find the index of the best particle
        gbest_index = np.argmin(pbest_fitness)
        
        # Update the position of the best particle
        gbest_position = pbest_position[gbest_index]
        
        ##print('Round', t+1, ', Best Position:', gbest_position, ', Cost:', pbest_fitness[gbest_index])
        ##generateMap(particles, pbest_fitness, real_position, t)

    # Print the results
    #print('Global Best Position: ', gbest_position)
    #print('Best Fitness Value: ', min(pbest_fitness))
    #print('Average Particle Best Fitness Value: ', np.average(pbest_fitness))
    #print('Number of Generation: ', t)
    
    return gbest_position

## TESTES

from google.colab import drive
drive.mount('/content/drive')

In [4]:
df = pd.read_csv('db_yuri_training5_semsala42_1-todos-aps-maxValue.csv')
#df = pd.read_csv('/content/drive/MyDrive/UFAM/Doutorado/Doutorado-Artigo3/db_yuri_training5_semsala42_1-todos-aps-maxValue.csv')
df = df[(df["DEVICE"] == '2055') | (df["DEVICE"] == '121B') | (df["DEVICE"] == '20B5')].reset_index(drop=True)

#df_walls = pd.read_csv('/content/drive/MyDrive/UFAM/Doutorado/Doutorado-Artigo3/walls_values.csv')
df_walls = pd.read_csv('walls_values.csv')

In [5]:
df_walls

,LABEL,X,Y,WAP1,WAP2,WAP3,WAP4,WAP5,WAP6,WAP7,WAP8,WAP9,WAP10,WAP11,WAP12,WAP13,WAP14,WAP15
0,16_1_5,1.5,14.8,1,3,4,3,5,6,5,4,3,2,0,1,1,3,5
1,16_1_3,1.5,12.5,1,3,6,4,5,6,5,4,3,2,0,1,1,3,4
2,16_1_1,1.5,10.2,1,2,6,6,4,5,5,4,3,2,0,1,1,1,4
3,16_3_5,3.5,14.8,1,2,4,3,5,6,5,4,3,2,0,1,1,3,5
4,16_3_3,3.5,12.5,1,3,4,3,5,6,5,4,3,2,0,1,1,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,63_4_1,11.4,9.5,2,1,2,2,2,3,4,3,2,1,1,2,0,0,2
144,63_5_7,12.7,14.9,4,3,4,4,5,5,4,3,2,1,1,3,0,2,4
145,63_5_5,12.7,13.1,4,3,4,4,5,5,4,3,2,1,1,2,0,2,4
146,63_5_3,12.7,11.3,4,1,2,4,4,5,4,3,2,1,1,2,0,2,3


In [7]:
df_walls.at[41, 'LABEL']

'41_5_7'

In [33]:
X = {"16_1_5": [0,1.5,14.8],
"16_1_3": [1,1.5,12.5],
"16_1_1": [2,1.5,10.2],
"16_3_5": [3,3.5,14.8],
"16_3_3": [4,3.5,12.5],
"16_3_1": [5,3.5,10.2],
"16_5_5": [6,5.5,14.8],
"16_5_3": [7,5.5,12.5],
"16_5_1": [8,5.5,10.2],
"16_7_5": [9,7.5,14.8],
"16_7_3": [10,7.5,12.5],
"16_7_1": [11,7.5,10.2],
"16_9_5": [12,9.5,14.8],
"16_9_3": [13,9.5,12.5],
"16_9_1": [14,9.5,10.2],
"39_1_5": [15,30.7,6.1],
"39_1_3": [16,30.7,3.8],
"39_1_1": [17,30.7,1.5],
"39_3_5": [18,32.9,6.1],
"39_3_3": [19,32.9,3.8],
"39_3_1": [20,32.9,1.5],
"39_5_5": [21,35.2,6.1],
"39_5_3": [22,35.2,3.8],
"39_5_1": [23,35.2,1.5],
"40_1_5": [24,16.6,6.1],
"40_1_3": [25,16.6,3.8],
"40_1_1": [26,16.6,1.5],
"40_3_5": [27,18.6,6.1],
"40_3_3": [28,18.6,3.8],
"40_3_1": [29,18.6,1.5],
"40_5_5": [30,20.6,6.1],
"40_5_3": [31,20.6,3.8],
"40_5_1": [32,20.6,1.5],
"41_1_7": [33,1.4,8.2],
"41_1_5": [34,1.4,6.0],
"41_1_3": [35,1.4,3.8],
"41_1_1": [36,1.4,1.6],
"41_3_7": [37,3.4,8.2],
"41_3_5": [38,3.4,6.0],
"41_3_3": [39,3.4,3.8],
"41_3_1": [40,3.4,1.6],
"41_5_7": [41,5.4,8.2],
"41_5_5": [42,5.4,6.0],
"41_5_3": [43,5.4,3.8],
"41_5_1": [44,5.4,1.6],
"43_1_5": [45,22.6,14.8],
"43_1_3": [46,22.6,12.5],
"43_1_1": [47,22.6,10.2],
"43_3_5": [48,24.6,14.8],
"43_3_3": [49,24.6,12.5],
"43_3_1": [50,24.6,10.2],
"43_5_5": [51,26.6,14.8],
"43_5_3": [52,26.6,12.5],
"43_5_1": [53,26.6,10.2],
"43_7_5": [54,28.6,14.8],
"43_7_3": [55,28.6,12.5],
"43_7_1": [56,28.6,10.2],
"45_1_5": [57,30.6,14.8],
"45_1_3": [58,30.6,12.5],
"45_1_1": [59,30.6,10.2],
"45_3_5": [60,32.6,14.8],
"45_3_3": [61,32.6,12.5],
"45_3_1": [62,32.6,10.2],
"45_5_5": [63,34.6,14.8],
"45_5_3": [64,34.6,12.5],
"45_5_1": [65,34.6,10.2],
"45_7_5": [66,36.6,14.8],
"45_7_3": [67,36.6,12.5],
"45_7_1": [68,36.6,10.2],
"46_1_5": [69,14.6,14.8],
"46_1_3": [70,14.6,12.5],
"46_1_1": [71,14.6,10.2],
"46_3_5": [72,16.6,14.8],
"46_3_3": [73,16.6,12.5],
"46_3_1": [74,16.6,10.2],
"46_5_5": [75,18.6,14.8],
"46_5_3": [76,18.6,12.5],
"46_5_1": [77,18.6,10.2],
"46_7_5": [78,20.6,14.8],
"46_7_3": [79,20.6,12.5],
"46_7_1": [80,20.6,10.2],
"47_1_5": [81,37.3,6.1],
"47_1_3": [82,37.3,3.8],
"47_1_1": [83,37.3,1.5],
"47_3_5": [84,39.3,6.1],
"47_3_3": [85,39.3,3.8],
"47_3_1": [86,39.3,1.5],
"47_5_5": [87,41.3,6.1],
"47_5_3": [88,41.3,3.8],
"47_5_1": [89,41.3,1.5],
"47_7_5": [90,43.3,6.1],
"47_7_3": [91,43.3,3.8],
"47_7_1": [92,43.3,1.5],
"51_1_7": [93,38.8,14.6],
"51_1_5": [94,38.8,12.5],
"51_1_3": [95,38.8,10.4],
"51_1_1": [96,38.8,8.2],
"51_3_7": [97,40.8,14.6],
"51_3_5": [98,40.8,12.5],
"51_3_3": [99,40.8,10.4],
"51_3_1": [100,40.8,8.2],
"51_5_7": [101,43.1,14.6],
"51_5_5": [102,43.1,12.5],
"51_5_3": [103,43.1,10.4],
"51_5_1": [104,43.1,8.2],
"53_1_5": [105,22.6,6.1],
"53_1_3": [106,22.6,3.8],
"53_1_1": [107,22.6,1.5],
"53_3_5": [108,24.6,6.1],
"53_3_3": [109,24.6,3.8],
"53_3_1": [110,24.6,1.5],
"53_5_5": [111,26.6,6.1],
"53_5_3": [112,26.6,3.8],
"53_5_1": [113,26.6,1.5],
"53_7_5": [114,28.6,6.1],
"53_7_3": [115,28.6,3.8],
"53_7_1": [116,28.6,1.5],
"57_1_6": [117,7.3,6.6],
"57_1_4": [118,7.3,4.9],
"57_1_2": [119,7.3,3.2],
"57_1_1": [120,7.3,1.6],
"57_2_6": [121,8.5,6.6],
"57_2_4": [122,8.5,4.9],
"57_2_2": [123,8.5,3.2],
"57_2_1": [124,8.5,1.6],
"61_2_1": [125,8.3,8.2],
"61_4_1": [126,10.3,8.2],
"61_6_1": [127,12.3,8.2],
"61_8_1": [128,14.3,8.2],
"61_10_1": [129,16.3,8.2],
"61_12_1": [130,18.3,8.2],
"61_14_1": [131,20.3,8.2],
"62_1_1": [132,22.6,7.9],
"62_3_1": [133,24.6,7.9],
"62_5_1": [134,26.6,7.9],
"62_7_1": [135,28.6,7.9],
"62_9_1": [136,30.6,7.9],
"62_11_1": [137,32.6,7.9],
"62_13_1": [138,34.6,7.9],
"62_15_1": [139,36.6,7.9],
"63_4_7": [140,11.4,14.9],
"63_4_5": [141,11.4,13.1],
"63_4_3": [142,11.4,11.3],
"63_4_1": [143,11.4,9.5],
"63_5_7": [144,12.7,14.9],
"63_5_5": [145,12.7,13.1],
"63_5_3": [146,12.7,11.3],
"63_5_1": [147,12.7,9.5]}

In [43]:
low = 100000
index = 10000

for k,v in X.items():
    point_sample = [v[1], v[2]]
    dist = euclidian_distance(point_sample, particle_position)
    
    if dist < low:
        low = dist
        index = v[0]
        
return index

16_1_5 [0, 1.5, 14.8] [1.5, 14.8]
16_1_3 [1, 1.5, 12.5] [1.5, 12.5]
16_1_1 [2, 1.5, 10.2] [1.5, 10.2]
16_3_5 [3, 3.5, 14.8] [3.5, 14.8]
16_3_3 [4, 3.5, 12.5] [3.5, 12.5]
16_3_1 [5, 3.5, 10.2] [3.5, 10.2]
16_5_5 [6, 5.5, 14.8] [5.5, 14.8]
16_5_3 [7, 5.5, 12.5] [5.5, 12.5]
16_5_1 [8, 5.5, 10.2] [5.5, 10.2]
16_7_5 [9, 7.5, 14.8] [7.5, 14.8]
16_7_3 [10, 7.5, 12.5] [7.5, 12.5]
16_7_1 [11, 7.5, 10.2] [7.5, 10.2]
16_9_5 [12, 9.5, 14.8] [9.5, 14.8]
16_9_3 [13, 9.5, 12.5] [9.5, 12.5]
16_9_1 [14, 9.5, 10.2] [9.5, 10.2]
39_1_5 [15, 30.7, 6.1] [30.7, 6.1]
39_1_3 [16, 30.7, 3.8] [30.7, 3.8]
39_1_1 [17, 30.7, 1.5] [30.7, 1.5]
39_3_5 [18, 32.9, 6.1] [32.9, 6.1]
39_3_3 [19, 32.9, 3.8] [32.9, 3.8]
39_3_1 [20, 32.9, 1.5] [32.9, 1.5]
39_5_5 [21, 35.2, 6.1] [35.2, 6.1]
39_5_3 [22, 35.2, 3.8] [35.2, 3.8]
39_5_1 [23, 35.2, 1.5] [35.2, 1.5]
40_1_5 [24, 16.6, 6.1] [16.6, 6.1]
40_1_3 [25, 16.6, 3.8] [16.6, 3.8]
40_1_1 [26, 16.6, 1.5] [16.6, 1.5]
40_3_5 [27, 18.6, 6.1] [18.6, 6.1]
40_3_3 [28, 18.6, 3.8] [18.6, 

In [32]:
for i in df_walls.itertuples():
    point_sample = [i.X, i.Y]
    print('"' +i.LABEL + '": [' + str(i.Index)+ "," + str(i.X) + "," + str(i.Y)+'],')

"16_1_5": [0,1.5,14.8],
"16_1_3": [1,1.5,12.5],
"16_1_1": [2,1.5,10.2],
"16_3_5": [3,3.5,14.8],
"16_3_3": [4,3.5,12.5],
"16_3_1": [5,3.5,10.2],
"16_5_5": [6,5.5,14.8],
"16_5_3": [7,5.5,12.5],
"16_5_1": [8,5.5,10.2],
"16_7_5": [9,7.5,14.8],
"16_7_3": [10,7.5,12.5],
"16_7_1": [11,7.5,10.2],
"16_9_5": [12,9.5,14.8],
"16_9_3": [13,9.5,12.5],
"16_9_1": [14,9.5,10.2],
"39_1_5": [15,30.7,6.1],
"39_1_3": [16,30.7,3.8],
"39_1_1": [17,30.7,1.5],
"39_3_5": [18,32.9,6.1],
"39_3_3": [19,32.9,3.8],
"39_3_1": [20,32.9,1.5],
"39_5_5": [21,35.2,6.1],
"39_5_3": [22,35.2,3.8],
"39_5_1": [23,35.2,1.5],
"40_1_5": [24,16.6,6.1],
"40_1_3": [25,16.6,3.8],
"40_1_1": [26,16.6,1.5],
"40_3_5": [27,18.6,6.1],
"40_3_3": [28,18.6,3.8],
"40_3_1": [29,18.6,1.5],
"40_5_5": [30,20.6,6.1],
"40_5_3": [31,20.6,3.8],
"40_5_1": [32,20.6,1.5],
"41_1_7": [33,1.4,8.2],
"41_1_5": [34,1.4,6.0],
"41_1_3": [35,1.4,3.8],
"41_1_1": [36,1.4,1.6],
"41_3_7": [37,3.4,8.2],
"41_3_5": [38,3.4,6.0],
"41_3_3": [39,3.4,3.8],
"41_3_1": [40,3.4

In [5]:
df_walls.at([0], 'WAP1')

TypeError: '_AtIndexer' object is not callable

In [ ]:
def attenuation():
    u = 0
    v = 0
    
    u = random.uniform(0, 1)
    v = random.uniform(0, 1)
    
    normal = math.sqrt(-2.0 * math.log(u)) * math.cos(2.0 * math.pi * v)
    return normal * STD_DEV


# Cada particula tem uma posição [x,y] e pegamos essa posição para obter as distâncias pros respectivos waps
# e através da distância obter o RSSI entre eles. No final será retornado um dicionário com o RSSI para todos os WAPS.
def particle_RSSI(particle_position):
    particle_sample = {}

    #label = sample_dfwall_low_dist(particle_position)
    
    for i in range(len(list_aps)):
        wap_position = aps_positions[list_aps[i]]
        dist = euclidian_distance(wap_position, particle_position)
        
        #walls_qtd = df_walls[df_walls["LABEL"] == label][list_aps[i]].iloc[0]
        #print(label, list_aps[i], walls_qtd)
        walls_qtd=0
        
        #N   = round(random.uniform(3.5, 4.5),1)
        #PL0 = round(random.uniform(50, 65),1)
        #WALL_LOSS = round(random.uniform(2, 6),1)
        
        N = 3.5
        PL0 = 60
        WALL_LOSS = 3
        
        if dist < 0.1:
            RSSI = -PL0
        else:
            distLoss = PL0 + 10 * N * math.log10(dist/d0)
            wallLoss = walls_qtd * WALL_LOSS
            pathLoss = distLoss + wallLoss
            
            #RSSI = round((TXPOWER - ( pathLoss)) + attenuation() ,0)
            RSSI = round((TXPOWER - ( pathLoss)) ,0)
        
        if RSSI < -95: RSSI = -105
        particle_sample[list_aps[i]] = RSSI

    return particle_sample

#RMSD entre os RSSI da particula e do sample
def fitness_function(particle_position, row):
    particula = particle_RSSI(particle_position)
    error = 0
    
    for i in range(len(list_aps)):
        error += math.pow((particula[list_aps[i]] - getattr(row, list_aps[i])) , 2)
    
    error = error

    return round(error,2)

In [ ]:
def error_by_room(room):
    error_by_room = []
    room_df     = df[df['ROOM_ID'] == room]
    room_points = room_df['LABEL'].unique()
    
    for point in room_points:
        point_df = room_df[room_df['LABEL'] == point]
        
        for row in point_df.itertuples():
            real_position = [row.X, row.Y]

            estimated_position = pso_2d(population, dimension, generation, fitness_criterion, real_position, row)
            estimated_error = euclidian_distance(real_position, estimated_position)
            
            error_by_room.append(estimated_error)
            
    return_error = np.mean(error_by_room)
    print('Sala:', room, 'Error:', return_error)
    return return_error

In [ ]:
list_rooms = list(df['ROOM_ID'].unique())

inicio_processo = time()

subprocessos = []
pool = Pool(15)

for room in tqdm(list_rooms):
#for room in list_rooms:
    #resultado_paralelo = error_by_room(room)
    resultado_paralelo = pool.apply_async(error_by_room, (room, ))
    subprocessos.append(resultado_paralelo)

lista_api_paralela = [result.get(timeout=120) for result in tqdm(subprocessos)]
print('\n\nMEDIA:', np.mean(lista_api_paralela))

fim_processo = time()
processamento_paralelo = fim_processo - inicio_processo
print('Processamento paralelo:', round( (processamento_paralelo), 1 ), 'segundos')